### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [9]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [7]:
scope = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "jupyter-integration-credentials-redistricting.json", scope
)
gc = gspread.authorize(credentials)

### read in data from google sheets

In [8]:
spreadsheet_key = "1ynudJpCbMdNpTan6buHptxLsm2q6GLRTB5gzXxnmRok"
book = gc.open_by_key(spreadsheet_key)

In [10]:
# Opening the worksheet by using Worksheet ID
workbook = gc.open_by_key(spreadsheet_key)
# Selecting which sheet to pulling the data
sheet = workbook.worksheet("TOTAL")
# Pulling the data and transform it to the data frame
values = sheet.get_all_values()

### clean data for jupyter notebooks

In [55]:
df = pd.DataFrame(values).iloc[1:].reset_index()
header = df.iloc[0]
header.iloc[1] = "state"
header.iloc[5] = "blank"

/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_3559/1030220485.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  header.iloc[1] = "state"
/var/folders/gg/6wnjn6cd1qlfst8mkw4g4xqc0000gn/T/ipykernel_3559/1030220485.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  header.iloc[5] = "blank"


In [56]:
df.columns = header

In [57]:
df = df.iloc[1:52]

In [63]:
df_clean = df.drop(columns=[df.columns[0]])

In [68]:
df_clean.columns = (
    df_clean.columns.str.lower()
    .str.replace(" ", "_")
    .str.replace(r"\(.*?\)", "", regex=True)
)

In [69]:
df_clean

### get data for map making

In [70]:
import us

In [77]:
all_features = []

for st in us.states.STATES:
    states_dict = {
        "fips": st.fips,
        "name": st.name,
        "abbr": st.abbr,
        "is_continental": st.is_continental,
        "statehood_year": st.statehood_year,
        "capital": st.capital,
        "capital_tz": st.capital_tz,
        "ap_abbr": st.ap_abbr,
        "shapefile_urls": st.shapefile_urls(),
    }
    all_features.append(states_dict)

In [74]:
name_to_ap = us.states.mapping("name", "ap_abbr")

In [82]:
gdf = (
    pd.DataFrame(all_features)
    .sort_values("fips", ascending=True)
    .reset_index(drop=True)
)

### I assume what this line is doing is going through the shapefile urls for the data

In [87]:
gdf[
    [
        "tract_shp",
        "cd_shp",
        "county_shp",
        "state_shp",
        "zcta_shp",
        "block_shp",
        "blockgroup_shp",
    ]
] = gdf["shapefile_urls"].apply(pd.Series)

In [95]:
import geopandas as gpd

In [118]:
### this will get the geography for an individual shape from the shapefile link
gpd.read_file(gdf["state_shp"][0])

,REGION10,DIVISION10,STATEFP10,STATENS10,GEOID10,STUSPS10,NAME10,LSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,3,6,01,01779775,01,AL,Alabama,00,G4000,A,131170787086,4596557530,+32.7396323,-086.8434593,"POLYGON ((-86.31127 34.99110, -86.30899 34.991..."


probably need to loop through gdf for shapefiles

In [ ]:
for st in state_shp:

    ax = gdf[gdf["year"] == y].boundary.plot(
        edgecolor="white", linewidth=0.2, figsize=(10, 7)
    )

    zones.plot(ax=ax, facecolor="#e9e9e9", edgecolor="#fff", linewidth=0.4)

    gdf[gdf["year"] == y].plot(
        ax=ax,
        column="days",
        legend=False,
        categorical=True,
        # scheme="natural_breaks",
        # k=7,
        cmap="Oranges",
        scheme="User_Defined",
        classification_kwds=dict(bins=breaks_manual),
    )

    states.plot(ax=ax, facecolor="none", edgecolor="#888", linewidth=0.4)

    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    for edge in ["right", "bottom", "top", "left"]:
        ax.spines[edge].set_visible(False)

    # ax.set_title(
    #     f"{y}",
    #     font="Summit Sans",
    #     size=20,
    #     weight="bold",
    # )

    plt.savefig(
        f"../../graphics/extreme-heat-maps-20220823/images/map_{y}.png",
        dpi=72,
        transparent=True,
        pad_inches=0,
        bbox_inches="tight",
    )